In [1]:
import qiskit
from qiskit_experiments.library.characterization import Tphi

In [2]:
# set the computation units to microseconds
unit = 'us' #microseconds
qubit = 0
# set the desired delays
delays_t1 = list(range(1, 40, 3))
delays_t2 = list(range(1, 50, 2))




In [3]:
exp = Tphi(qubit=0, delays_t1=delays_t1, delays_t2=delays_t2, unit="s", osc_freq=0.1)
    

In [4]:
print(exp._experiments[1].circuits()[0])

     ┌───┐┌─────────────┐┌─────────┐ ░ ┌───┐ ░ ┌─┐
q_0: ┤ H ├┤ Delay(1[s]) ├┤ Rz(π/5) ├─░─┤ H ├─░─┤M├
     └───┘└─────────────┘└─────────┘ ░ └───┘ ░ └╥┘
c: 1/═══════════════════════════════════════════╩═
                                                0 


In [5]:
print(exp.num_experiments)

2


In [6]:

from qiskit_experiments.test.tphi_backend import TphiBackend
backend = TphiBackend(t1=10, t2ramsey=25, freq=0.1, dt_factor=1)
#print(backend)
expdata = exp.run(backend, experiment_data=None)
#print(expdata)

backend = tphi_simulator
t1_shots = 1024


AttributeError: 'NoneType' object has no attribute 'job_id'